In [4]:
import weaviate
from weaviate.classes.config import Configure

# Step 1.1: Connect to your local Weaviate instance
with weaviate.connect_to_local(port=8090) as client:
    # Step 1.2: Create a collection
    movies = client.collections.create(
        name="Movie",
        vector_config=Configure.Vectors.text2vec_ollama(  # Configure the Ollama embedding integration
            api_endpoint="http://ollama:11434",  # If using Docker you might need: http://host.docker.internal:11434
            model="nomic-embed-text",  # The model to use
        ),
    )

    # Step 1.3: Import three objects
    data_objects = [
        {"title": "The Matrix", "description": "A computer hacker learns about the true nature of reality and his role in the war against its controllers.", "genre": "Science Fiction"},
        {"title": "Spirited Away", "description": "A young girl becomes trapped in a mysterious world of spirits and must find a way to save her parents and return home.", "genre": "Animation"},
        {"title": "The Lord of the Rings: The Fellowship of the Ring", "description": "A meek Hobbit and his companions set out on a perilous journey to destroy a powerful ring and save Middle-earth.", "genre": "Fantasy"},
    ]

    movies = client.collections.use("Movie")
    with movies.batch.fixed_size(batch_size=200) as batch:
        for obj in data_objects:
            batch.add_object(properties=obj)

    print(f"Imported & vectorized {len(movies)} objects into the Movie collection")

Imported & vectorized 3 objects into the Movie collection


In [ ]:
# semantic vector search
import weaviate
import json

# Step 2.1: Connect to your local Weaviate instance
with weaviate.connect_to_local(port=8090) as client:

    # Step 2.2: Use this collection
    movies = client.collections.use("Movie")

    # Step 2.3: Perform a semantic search with NearText
    response = movies.query.near_text(
        query="sci-fi",
        limit=2  #how many objects it will
    )

    for obj in response.objects:
        print(json.dumps(obj.properties, indent=2))  # Inspect the results

{
  "title": "The Matrix",
  "description": "A computer hacker learns about the true nature of reality and his role in the war against its controllers.",
  "genre": "Science Fiction"
}
{
  "title": "The Lord of the Rings: The Fellowship of the Ring",
  "description": "A meek Hobbit and his companions set out on a perilous journey to destroy a powerful ring and save Middle-earth.",
  "genre": "Fantasy"
}


In [6]:
#RAG
import weaviate
from weaviate.classes.generate import GenerativeConfig

# Step 2.1: Connect to your local Weaviate instance
with weaviate.connect_to_local(port=8090) as client:

    # Step 2.2: Use this collection
    movies = client.collections.use("Movie")

    # Step 2.3: Perform RAG with on NearText results
    response = movies.generate.near_text(
        query="sci-fi",
        limit=1,
        grouped_task="Write a tweet with emojis about this movie.",
        generative_provider=GenerativeConfig.ollama(  # Configure the Ollama generative integration
            api_endpoint="http://ollama:11434",  # If using Docker you might need: http://host.docker.internal:11434
            model="llama3.2",  # The model to use
        ),
    )

    print(response.generative.text)  # Inspect the results

"Just watched #TheMatrix 🤖💻 and my mind is BLOWN! 😲 Neo's journey to uncover the truth behind the simulated reality will leave you questioning everything 🤔 What's real, what's not? The answer might just change your life 🎥 #WachowskiSiblings #SciFiClassic"


Read from external data

In [2]:
import pandas as pd

df = pd.read_csv('jeopardy_questions.csv',nrows=100)
df.head()

,show_number,air_date,round,category,value,question,answer
0,4680,12/31/04,Jeopardy!,HISTORY,200.0,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,12/31/04,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200.0,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,12/31/04,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200.0,The city of Yuma in this state has a record av...,Arizona
3,4680,12/31/04,Jeopardy!,THE COMPANY LINE,200.0,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,12/31/04,Jeopardy!,EPITAPHS & TRIBUTES,200.0,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [3]:
import weaviate
import pandas as pd
from weaviate.classes.config import Configure

# Step 1.1: Connect to your local Weaviate instance
with weaviate.connect_to_local(port=8090) as client:

    # Step 1.2: Create a collection
    jeopardy = client.collections.create(
        name="JeopardyQuestion",
        vector_config=Configure.Vectors.text2vec_ollama(
            api_endpoint="http://ollama:11434",
            model="nomic-embed-text",
        ),
    )

    # Step 1.4: Import data rows
    jeopardy = client.collections.use("JeopardyQuestion")

    with jeopardy.batch.fixed_size(batch_size=200) as batch:
        for _, row in df.iterrows():
            batch.add_object(
                properties={
                    "question": row["question"],
                    "answer": row["answer"],
                    "category": row["category"],
                }
            )

    print(f"Imported & vectorized {len(df)} objects into the JeopardyQuestion collection")


Imported & vectorized 100 objects into the JeopardyQuestion collection


In [37]:
#access imported data from weaviate
with weaviate.connect_to_local(port=8090) as client:
    jeopardy= client.collections.use("JeopardyQuestion")
    results = jeopardy.query.fetch_objects(limit=5)

for obj in results.objects:
    print(obj.properties)
    print("----")


{'answer': 'correct', 'question': 'As an adjective, it can mean proper; as a verb, "to grade papers"', 'category': 'IN THE DICTIONARY'}
----
{'answer': 'Jackie Gleason', 'question': '"And away we go"', 'category': 'EPITAPHS & TRIBUTES'}
----
{'answer': 'the UV index', 'question': "On June 28, 1994 the nat'l weather service began issuing this index that rates the intensity of the sun's radiation", 'category': 'EVERYBODY TALKS ABOUT IT...'}
----
{'answer': 'Michael Jordan', 'question': 'No. 8: 30 steals for the Birmingham Barons; 2,306 steals for the Bulls', 'category': "ESPN's TOP 10 ALL-TIME ATHLETES"}
----
{'answer': 'imp', 'question': 'A small demon, or a mischievous child (who might be a little demon!)', 'category': '3-LETTER WORDS'}
----


In [38]:
# delete data against a particular collection
from weaviate.classes.query import Filter

with weaviate.connect_to_local(port=8090) as client:

    jeopardy = client.collections.use("JeopardyQuestion")
    result = jeopardy.data.delete_many(
        #where=Filter.by_property("answer").like("plane*"),
        where=Filter.by_property("answer").like("*"),
        dry_run=True, #make False to perform delete operation 
        verbose=True
    )

    print(result)

DeleteManyReturn(failed=0, matches=100, objects=[DeleteManyObject(uuid=_WeaviateUUIDInt('d8022fa3-9431-4e72-9481-547428402a92'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('eb017d34-f222-4c7d-9ceb-c24c4087b968'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('e8e35b80-d871-42eb-b333-eef6cb6d7a80'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('abe37aaf-3695-48c2-abad-263faecb640a'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('9f04a8e9-c2c0-4269-808f-0673e2a91772'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('6d7631ee-ac24-4dc1-ab95-02ad0d5f43ee'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('72e9579b-7d27-46f1-9c45-7e8c74c77b30'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('025cd21f-c65a-4f8f-bb82-7fae8dd95ccd'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('129c9d75-4507-4768-b336-10f729e

In [ ]:
# delete the entire collection

import weaviate

with weaviate.connect_to_local(port=8090) as client:

    collection_name = "JeopardyQuestion"
    try:
        client.collections.delete("JeopardyQuestion")
        print(f'{collection_name} has been removed succesfully')
    except Exception as e:
        print(str(e))

JeopardyQuestion has been removed succesfully


In [41]:
#access all collections

with weaviate.connect_to_local(port=8090) as client:
    result = client.collections.list_all()
    print(result.keys())
    print('*'*100)

    config = result['JeopardyQuestion']

    print(config)


dict_keys(['JeopardyQuestion', 'Movie'])
****************************************************************************************************
_CollectionConfigSimple(name='JeopardyQuestion', description=None, generative_config=None, properties=[_Property(name='question', description="This property was generated by Weaviate's auto-schema feature on Thu Nov 20 12:37:23 2025", data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-ollama': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)}), _Property(name='answer', description="This property was generated by Weaviate's auto-schema feature on Thu Nov 20 12:37:23 2025", data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'w

In [42]:
#get the items count

with weaviate.connect_to_local(port=8090) as client:
    jeopardy = client.collections.get("JeopardyQuestion")

    result = jeopardy.aggregate.over_all().total_count

    print(result)

#client.query.aggregate("JeopardyQuestion").with_meta_count().do()

100


In [ ]:
#Fetch objects WITH their vectors

with weaviate.connect_to_local(port=8090) as client:
    jeopardy = client.collections.use("JeopardyQuestion")

    k=0
    for item in jeopardy.iterator(include_vector=True):
        print(item.properties)
        print(item.vector)
        print('*'*100)
        k+=1
        if k>4:
            break    


{'answer': 'correct', 'question': 'As an adjective, it can mean proper; as a verb, "to grade papers"', 'category': 'IN THE DICTIONARY'}
{'default': [0.04855242744088173, 0.0033910414204001427, -0.1551145762205124, -0.03973388671875, 0.05596520006656647, 0.0467025525867939, -0.019675739109516144, 0.008045637048780918, 0.02822018601000309, 0.03429902717471123, -0.001427136012353003, 0.01384754292666912, 0.05410240218043327, 0.025519775226712227, 0.019152861088514328, -0.047667283564805984, 0.022316668182611465, -0.07565690577030182, 0.010372313670814037, -0.03013365902006626, -0.047310687601566315, -0.04713110998272896, -0.01748511753976345, 0.02407144382596016, 0.07174593210220337, 0.005288455635309219, -0.05652543902397156, -0.030082495883107185, -0.04155031591653824, 0.008176485076546669, -0.029267290607094765, -0.0038369009271264076, 0.025322316214442253, -0.02110113948583603, -0.02671159617602825, -0.06417999416589737, 0.01814148761332035, 0.061063025146722794, -0.03822052478790283,

In [76]:
# semantic vector search
from weaviate.classes.query import MetadataQuery
import weaviate
import json

# Step 2.1: Connect to your local Weaviate instance
with weaviate.connect_to_local(port=8090) as client:

    # Step 2.2: Use this collection
    jeopardy = client.collections.use("JeopardyQuestion")

    # Step 2.3: Perform a semantic search with NearText
    response = jeopardy.query.near_text(
        query="sci-fi",
        limit=2,  #how many objects it will
        return_properties=["question", "answer"],
        certainty=0.25, # max accepted distance 
        return_metadata=MetadataQuery(distance=True)
    )

    for obj in response.objects:
        print(json.dumps(obj.properties, indent=2))  # Inspect the results
        print(f'<<{obj.uuid}>>')
        print(obj.metadata.distance)
        print('*'*100)

{
  "answer": "photosynthesis",
  "question": "During this plant process, carbon dioxide & water combine with light energy to create oxygen & glucose"
}
<<9d27c3d0-73f6-40ec-94fb-3091473db306>>
0.48442983627319336
****************************************************************************************************
{
  "answer": "Icarus",
  "question": "Objects that pass closer to the sun than Mercury have been named for this mythological figure"
}
<<23489a58-7080-499a-9291-e6eb579ba393>>
0.4853353500366211
****************************************************************************************************


In [89]:
#RAG
import weaviate
from weaviate.classes.generate import GenerativeConfig

with weaviate.connect_to_local(port=8090) as client:
    jeopardy = client.collections.use("JeopardyQuestion")

    result = jeopardy.generate.near_text(
        query="animals",
        limit=1,
        return_properties=["question", "answer"],
        grouped_task="Generate a question to which the answer is {{answer}}.",
        generative_provider=GenerativeConfig.ollama(
            api_endpoint="http://ollama:11434",
            model="llama3.2"
        )
    )

    print("Generated:", result.generative.text)
    #print("Source object:", result.objects[0].properties)


Generated: Here's a question with the answer "{a kennel}":

What is typically used to transport dogs in the airline industry?
